<a href="https://colab.research.google.com/github/icecat14159/PL-Repo./blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80HW02_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread gradio pandas --quiet

In [42]:
# ---------------- 模組導入 ----------------
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import gradio as gr
import google.generativeai as genai
from datetime import datetime

In [43]:
# ---------------- 授權 Google Sheet ----------------
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sheet_url = "https://docs.google.com/spreadsheets/d/1NKTJvXrINjsxb3I80B_2wYTDcc-wRDMosUcD0P4qlVo/edit?usp=sharing"
worksheet = gc.open_by_url(sheet_url).sheet1

In [46]:
# ---------------- Gemini 設定 ----------------
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('models/gemini-1.5-flash-latest')

In [47]:
# ---------------- Gemini 測試 ----------------
test = gemini_model.generate_content("你好")
print(test.text)

KeyboardInterrupt: 

In [48]:
# ---------------- 功能定義 ----------------
def read_data():
    """讀取目前資料"""
    df = pd.DataFrame(worksheet.get_all_records())
    if df.empty:
        return "目前無資料"
    return df

def add_record(name, sid, hw1, hw2, mid, fin):
    """新增學生成績，並自動計算總分與平均"""
    try:
        scores = [float(hw1), float(hw2), float(mid), float(fin)]
        total = sum(scores)
        avg = total / len(scores)
    except ValueError:
        return "成績必須為數字"

    new_row = [name, sid, hw1, hw2, mid, fin, total, round(avg, 2)]
    worksheet.append_row(new_row)
    return f"已新增 {name}（學號:{sid}），平均 {round(avg,2)} 分"

def delete_record(index):
    """刪除指定筆資料"""
    df = pd.DataFrame(worksheet.get_all_records())
    if df.empty:
        return "無資料可刪除"
    if index < 1 or index > len(df):
        return "索引超出範圍"
    worksheet.delete_rows(index + 1)
    return f"已刪除第 {index} 筆資料"

def modify_record(index, name, sid, hw1, hw2, mid, fin):
    """修改指定筆資料"""
    df = pd.DataFrame(worksheet.get_all_records())
    if df.empty:
        return "無資料可修改"
    if index < 1 or index > len(df):
        return "索引超出範圍"
    try:
        scores = [float(hw1), float(hw2), float(mid), float(fin)]
        total = sum(scores)
        avg = total / len(scores)
    except ValueError:
        return "成績必須為數字"
    worksheet.update(f"A{index+1}:H{index+1}", [[name, sid, hw1, hw2, mid, fin, total, round(avg,2)]])
    return f"已修改第 {index} 筆資料"

def ai_summary():
    """使用 Gemini 產生成績總結，並寫回試算表"""
    df = pd.DataFrame(worksheet.get_all_records())
    if df.empty:
        return "目前無資料"
    text_data = df.to_string(index=False)
    prompt = f"以下是學生的成績資料（姓名、學號、作業1、作業2、期中、期末、總分、平均）。請以中文給出整體分析與趨勢總結：\n\n{text_data}"
    response = gemini_model.generate_content(prompt)
    summary = response.text
    worksheet.append_row(["AI 總結", "", "", "", "", "", "", summary])
    return summary

In [49]:
# ---------------- Gradio 介面 ----------------
with gr.Blocks() as demo:
    gr.Markdown("## 📘 作業2：成績一本通（含 Gemini AI 總結）")

    with gr.Tab("查看資料"):
        btn_view = gr.Button("重新讀取")
        out_view = gr.DataFrame()
        btn_view.click(read_data, outputs=out_view)

    with gr.Tab("新增資料"):
        name_in = gr.Textbox(label="學生姓名")
        sid_in = gr.Textbox(label="學生學號")
        hw1_in = gr.Number(label="作業1")
        hw2_in = gr.Number(label="作業2")
        mid_in = gr.Number(label="期中考")
        fin_in = gr.Number(label="期末考")
        btn_add = gr.Button("新增")
        out_add = gr.Textbox(label="結果")
        btn_add.click(add_record, inputs=[name_in, sid_in, hw1_in, hw2_in, mid_in, fin_in], outputs=out_add)

    with gr.Tab("修改資料"):
        idx_m = gr.Number(label="第幾筆（從1開始）", precision=0)
        name_m = gr.Textbox(label="學生姓名")
        sid_m = gr.Textbox(label="學生學號")
        hw1_m = gr.Number(label="作業1")
        hw2_m = gr.Number(label="作業2")
        mid_m = gr.Number(label="期中考")
        fin_m = gr.Number(label="期末考")
        btn_mod = gr.Button("修改")
        out_mod = gr.Textbox(label="結果")
        btn_mod.click(modify_record, inputs=[idx_m, name_m, sid_m, hw1_m, hw2_m, mid_m, fin_m], outputs=out_mod)

    with gr.Tab("刪除資料"):
        idx_d = gr.Number(label="第幾筆（從1開始）", precision=0)
        btn_del = gr.Button("刪除")
        out_del = gr.Textbox(label="結果")
        btn_del.click(delete_record, inputs=idx_d, outputs=out_del)

    with gr.Tab("AI 總結"):
        btn_sum = gr.Button("產生總結")
        out_sum = gr.Textbox(label="Gemini 總結結果", lines=6)
        btn_sum.click(ai_summary, outputs=out_sum)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c849d949182d3d415.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
